# Spark Practical Work

We are supposed to create a model capable of predicting the arrival delay time of a commercial flight based on several parameters known at the take-off time. Tasks:
* Load the input data, previously stored at a known location.
* Select, process and transform the input variables, to prepare them for training the model.
* Perform some basic analysis of each input variable. 
* Create a ML model that predicts the arrival delay time.
* Validate the created model and provide some measures of its accuracy.


In [1]:
import os
os.getcwd()

'/home/dslab/workspaces/rrunix/spark/final_project'

# 1. Load data

In [2]:
# extract files into csv formats
import bz2
# extract every bz2 format file in ../BigData/data/project_data/ to csv file
files = os.listdir("../BigData/data/project_data/")
def bz2_to_csv(files):
	for file in files:
		if file.endswith(".bz2"):
			file_path = "../BigData/data/project_data/" + file
			with bz2.open(file_path, "rb") as f:
				file_content = f.read()
			with open("../BigData/data/project_data/" + file[:-4], "wb") as f:
				f.write(file_content)

bz2_to_csv(files)

In [8]:
# Create a SparkSession
from pyspark.sql import SparkSession
from pyspark import SparkContext

sc = SparkContext("local", "ComercialFlights")
spark = SparkSession.builder \
            .appName("First Session") \
            .master("local[*]") \
            .getOrCreate()

sc.setLogLevel("ERROR")
print("Spark Version: {}".format(sc.version))

Spark Version: 3.5.0


In [9]:
# Use DataFrames to read csv files
# read all csv files in ../BigData/data/project_data/ to pyspark dataframe
csv_files = os.listdir("../BigData/data/project_data/")
def csv_to_df(csv_files):
	df_pyspark =[]
	for file in csv_files:
		file_path = "../BigData/data/project_data/" + file
		df = spark.read.csv(file_path, header=True, inferSchema=True)
		df_pyspark.append(df)
	return df_pyspark

df_pyspark = csv_to_df(csv_files)

# unir todos los dataframes en uno solo
from functools import reduce
from pyspark.sql import DataFrame
def unionAll(*dfs):
	return reduce(DataFrame.unionAll, dfs)

df = unionAll(*df_pyspark)
df.show(5)

		
	

ERROR:root:KeyboardInterrupt while sending command.                 (0 + 1) / 1]
Traceback (most recent call last):
  File "/home/dslab/anaconda3/envs/spark/lib/python3.8/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/dslab/anaconda3/envs/spark/lib/python3.8/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/home/dslab/anaconda3/envs/spark/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [21]:
# comprobar el numero de filas de df 
print(df.count())
# comprobar el numero de columnas de df
print(len(df.columns))

25952068
29


# 2. Process data

The dataset has 29 columns. We won't use all of them. The ones that should be droped are: 
* ArrTime
* ActualElapsedTime
* AirTime
* TaxiIn
* Diverted
* CarrierDelay
* WeatherDelay
* NASDelay
* SecurityDelay
* LateAircraftDelay

Meaning of the variables: 
1. Year 1987-2008 
2. Month 1-12 
3. DayofMonth 1-31 
4. DayOfWeek 1 (Monday) - 7 (Sunday)
5. DepTime actual departure time (local, hhm m) 
6. CRSDepTime scheduled departure time (local, hhmm) 
7. CRSArrTime scheduled arrival time (local, hhmm) 
8. UniqueCarrier Airline code 
9. FlightNum flight number 
10. TailNum plane tail number 
11. CRSElapsedTime in minutes (estimated flight time)
12. ArrDelay arrival delay, in minutes -- TARGET VARIABLE
13. DepDelay departure delay, in minutes 
14. Origin origin IATA airport code 
15. Dest destination IATA airport code 
16. Distance in miles 
17. TaxiOut taxi out time in minutes (tiempo que tarda el avión desde la puerta de embarque hasta el despegue")
18. Cancelled was the flight cancelled? 
19. CancellationCode reason for cancellation (A = carrier, B = weather, C = NAS, D = security) 

In [5]:
from pyspark.sql.functions import *
# rename columns:
def edit_column_names(df):
    df =  df.withColumnRenamed('DayofMonth','day_of_month').\
                withColumnRenamed('DayOfWeek','day_of_week').\
                withColumnRenamed('DepTime','actual_departure_time').\
                withColumnRenamed('CRSDepTime','scheduled_departure_time').\
                withColumnRenamed('ArrTime','actual_arrival_time').\
                withColumnRenamed('CRSArrTime','scheduled_arrival_time').\
                withColumnRenamed('UniqueCarrier','airline_code').\
                withColumnRenamed('FlightNum','flight_number').\
                withColumnRenamed('TailNum','plane_number').\
                withColumnRenamed('ActualElapsedTime','actual_flight_time').\
                withColumnRenamed('CRSElapsedTime','scheduled_flight_time').\
                withColumnRenamed('AirTime','air_time').\
                withColumnRenamed('ArrDelay','arrival_delay').\
                withColumnRenamed('DepDelay','departure_delay').\
                withColumnRenamed('TaxiIn','taxi_in').\
                withColumnRenamed('TaxiOut','taxi_out').\
                withColumnRenamed('CancellationCode','cancellation_code').\
                withColumnRenamed('CarrierDelay','carrier_delay').\
                withColumnRenamed('WeatherDelay','weather_delay').\
                withColumnRenamed('NASDelay','nas_delay').\
                withColumnRenamed('SecurityDelay','security_delay').\
                withColumnRenamed('LateAircraftDelay','late_aircraft_delay')
    for col in df.columns:
        df = df.withColumnRenamed(col, col.lower())
    return df

# select columns:
def my_columns (df):
    df = df.select('year','month','day_of_month', 'day_of_week', 'actual_departure_time', 'scheduled_departure_time', 'scheduled_arrival_time', 'airline_code', 'flight_number', 'plane_number', 'scheduled_flight_time', 'arrival_delay', 'departure_delay', 'origin', 'dest', 'distance', 'taxi_out', 'cancelled', 'cancellation_code')
    return df

# convert days to names:
def convert_days_to_names(df):
    df = df.withColumn('day_of_week', when(df.day_of_week == 1,'Monday').\
                                              when(df.day_of_week ==2,'Tuesday').\
                                              when(df.day_of_week ==3,'Wednesday').\
                                              when(df.day_of_week ==4,'Thursday').\
                                              when(df.day_of_week ==5,'Friday').\
                                              when(df.day_of_week ==6,'Saturday').\
                                              when(df.day_of_week ==7,'Sunday'))
    return df

# combine to create dates:
def add_date_column(df):
    df = df.withColumn('date', to_date(concat(col('day_of_month'), lit(' '), col('month'), lit(' '), col('year')), 'd M yyyy'))
    return df

# convert time to minutes:
def convert_time_to_minutes(df):
    # Define the external function
    def my_function(value)->int:
        value = str(value)
        if not value.isnumeric():
            return None
        value = str(int(value)) 
        mins = 0
        if value:
            if len(value) == 4:
                mins = int(value[:2])*60+int(value[2:])
            elif len(value) == 3:
                mins = int(value[:1])*60 + int(value[1:])
            return mins
        else:
            None
    # Register the UDF
    my_udf = udf(my_function, IntegerType())
    df = df.withColumn('actual_departure_time',my_udf(col('actual_departure_time')))\
                       .withColumn('scheduled_departure_time',my_udf(col('scheduled_departure_time')))\
                       .withColumn('actual_arrival_time',my_udf(col('actual_arrival_time')))\
                       .withColumn('scheduled_arrival_time',my_udf(col('scheduled_arrival_time')))
    return df

# standarize df
def standarize_dataframe(df):
    temp = edit_column_names(df)
    temp = my_columns(temp)
    temp = convert_days_to_names(temp)
    temp = add_date_column(temp)
    temp = convert_time_to_minutes(temp)
    return temp



In [6]:
new_df = standarize_dataframe(df)
new_df.show(5)

+----+-----+------------+-----------+---------------------+------------------------+-------------------+----------------------+------------+-------------+------------+------------------+---------------------+--------+-------------+---------------+------+----+--------+-------+--------+---------+-----------------+--------+-------------+-------------+---------+--------------+-------------------+----------+
|year|month|day_of_month|day_of_week|actual_departure_time|scheduled_departure_time|actual_arrival_time|scheduled_arrival_time|airline_code|flight_number|plane_number|actual_flight_time|scheduled_flight_time|air_time|arrival_delay|departure_delay|origin|dest|distance|taxi_in|taxi_out|cancelled|cancellation_code|diverted|carrier_delay|weather_delay|nas_delay|security_delay|late_aircraft_delay|      date|
+----+-----+------------+-----------+---------------------+------------------------+-------------------+----------------------+------------+-------------+------------+------------------+

In [22]:
# Filter data: We won't use all the columns, so we can drop them out
my_df = df.select("Year", "Month", "DayofMonth", "DayOfWeek", "DepTime", "CRSDepTime", "CRSArrTime", "UniqueCarrier", "FlightNum", "TailNum", "CRSElapsedTime", "ArrDelay", "DepDelay", "Origin", "Dest", "Distance", "TaxiOut", "Cancelled", "CancellationCode")

# print number of columns in df_pyspark and my_df
print(len(df.columns))
print(len(my_df.columns))

29
19


In [23]:
my_df.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- CRSElapsedTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: integer (nullable = true)
 |-- CancellationCode: string (nullable = true)



In [24]:
my_df.show(5)

+----+-----+----------+---------+-------+----------+----------+-------------+---------+-------+--------------+--------+--------+------+----+--------+-------+---------+----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|CRSElapsedTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiOut|Cancelled|CancellationCode|
+----+-----+----------+---------+-------+----------+----------+-------------+---------+-------+--------------+--------+--------+------+----+--------+-------+---------+----------------+
|1988|    1|         9|        6|   1348|      1331|      1435|           PI|      942|     NA|            64|      23|      17|   SYR| BWI|     273|     NA|        0|              NA|
|1988|    1|        10|        7|   1334|      1331|      1435|           PI|      942|     NA|            64|       8|       3|   SYR| BWI|     273|     NA|        0|              NA|
|1988|    1|        11|        1|   1446|      1331|      1435|           P

In [25]:
# contar número de variables nulas en cada columna
from pyspark.sql.functions import isnan, when, count, col
my_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in my_df.columns]).show()


+----+-----+----------+---------+-------+----------+----------+-------------+---------+-------+--------------+--------+--------+------+----+--------+-------+---------+----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|CRSElapsedTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiOut|Cancelled|CancellationCode|
+----+-----+----------+---------+-------+----------+----------+-------------+---------+-------+--------------+--------+--------+------+----+--------+-------+---------+----------------+
|   0|    0|         0|        0|      0|         0|         0|            0|        0|  84904|             0|       0|       0|     0|   0|       0|      0|        0|         4649550|
+----+-----+----------+---------+-------+----------+----------+-------------+---------+-------+--------------+--------+--------+------+----+--------+-------+---------+----------------+



no hay valores nulos

In [17]:
# contar el número de variables nan en cada columna
my_df.select([count(when(col(c) == "NA", c)).alias(c) for c in my_df.columns]).show()

+----+-----+----------+---------+-------+----------+----------+-------------+---------+--------+--------------+--------+--------+------+----+--------+--------+---------+----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|CRSArrTime|UniqueCarrier|FlightNum| TailNum|CRSElapsedTime|ArrDelay|DepDelay|Origin|Dest|Distance| TaxiOut|Cancelled|CancellationCode|
+----+-----+----------+---------+-------+----------+----------+-------------+---------+--------+--------------+--------+--------+------+----+--------+--------+---------+----------------+
|   0|    0|         0|        0| 139696|         0|         0|            0|        0|13027844|             0|  176198|  139696|     0|   0|   24234|13027844|        0|        13027844|
+----+-----+----------+---------+-------+----------+----------+-------------+---------+--------+--------------+--------+--------+------+----+--------+--------+---------+----------------+



In [26]:
# los valores nulos de "TailNum" cambiar por el valor más repetido en esa aerolínea (UniqueCarrier)
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, col
from pyspark.sql.functions import lit

# para cada aerolínea, calcular el valor más repetido en la columna "TailNum"
w = Window.partitionBy("UniqueCarrier").orderBy(col("count").desc())
df_tailnum = my_df.groupBy("UniqueCarrier", "TailNum").count().withColumn("rank", rank().over(w)).filter(col("rank") == 1).drop("rank", "count")
df_tailnum.show(5)

+-------------+-------+
|UniqueCarrier|TailNum|
+-------------+-------+
|           9E|   NULL|
|           AA|     NA|
|           AQ| N836AL|
|           AS|     NA|
|           B6| N281JB|
+-------------+-------+
only showing top 5 rows



In [10]:
spark.stop()